# Data Engineering Project | August 2022 | Victims Dataset
## Development Notebook

In [126]:
import pandas as pd
import numpy as np

### Load CSV data

In [127]:
acc_deaths_df = pd.read_csv('../data/accidental_deaths_2021_to_date.csv', header=0)
acc_deaths_df.name = 'Accidental Deaths 2021 To Date'

acc_injuries_df = pd.read_csv('../data/accidental_injuries_2021_to_date.csv', header=0)
acc_injuries_df.name = 'Accidental Injuries 2021 To Date'

chi_killed_df = pd.read_csv('../data/children_killed_2021_to_date.csv', header=0)
chi_killed_df.name = 'Children Killed 2021 To Date'

chi_injured_df = pd.read_csv('../data/children_injured_2021_to_date.csv', header=0)
chi_injured_df.name = 'Children Injured 2021 To Date'

teens_killed_df = pd.read_csv('../data/teens_killed_2021_to_date.csv', header=0)
teens_killed_df.name = 'Teens Killed 2021 To Date'

teens_injured_df = pd.read_csv('../data/teens_injured_2021_to_date.csv', header=0)
teens_injured_df.name = 'Teens Injured 2021 To Date'

ms_inj_kil_df = pd.read_csv('../data/mass_shootings_Injured_killed_2021_to_date.csv', header=0)
ms_inj_kil_df.name = 'Mass Shootings Injured Killed 2021 To Date'

In [128]:
all_input_dfs = [acc_deaths_df, acc_injuries_df, chi_killed_df,
                 chi_injured_df, teens_killed_df, teens_injured_df,
                 ms_inj_kil_df]

### Create new dataframe containing all records from all CSVs

1. Create new dataframe with no records
2. Iterate over each of the dataframes
3. Drop unnecessary columns for each of them (Operations column in this case)
4. Add new columns and values based on the names of the files

In [129]:
for df in all_input_dfs:
    
    # Set the Incident ID column as the index of the DF
    # df.set_index('Incident ID', inplace=True)
    
    # Drop "Operations" column since it doesn't offer any value
    df.drop('Operations', axis=1, inplace=True)

    # Add new "Children Involved" column to specify if the data has to do with
    # children
    if 'children' in df.name.lower():
        df.insert(7, 'Children Involved', True)
    else:
        df.insert(7, 'Children Involved', np.nan)

    # Add new "Teens Involved" column to specify if the data has to do with
    # children
    if 'teens' in df.name.lower():
        df.insert(8, 'Teens Involved', True)
    else:
        df.insert(8, 'Teens Involved', np.nan)

    # Add new "Accident" column to specify if the data has to do with accidents
    if 'accident' in df.name.lower():
        df.insert(9, 'Accident', True)
    else:
        df.insert(9, 'Accident', np.nan)

    # Add new "Mass Shooting" column to specify if the data has to do with
    # mass shootings
    if 'mass shooting' in df.name.lower():
        df.insert(10, 'Mass Shooting', True)
    else:
        df.insert(10, 'Mass Shooting', np.nan)

In [130]:
# Get a reference of the existing data structure
acc_deaths_df.dtypes

Incident ID            int64
Incident Date         object
State                 object
City Or County        object
Address               object
# Killed               int64
# Injured              int64
Children Involved    float64
Teens Involved       float64
Accident                bool
Mass Shooting        float64
dtype: object

In [131]:
all_victims_df = pd.DataFrame(
    {
        'Incident ID': pd.Series(dtype='int16'),
        'Incident Date': pd.Series(dtype='datetime64[ns]'),
        'State': pd.Series(dtype='object'),
        'City Or County': pd.Series(dtype='object'),
        'Address': pd.Series(dtype='object'),
        '# Killed': pd.Series(dtype='int8'),
        '# Injured': pd.Series(dtype='int8'),
        'Children Involved': pd.Series(dtype='bool'),
        'Teens Involved': pd.Series(dtype='bool'),
        'Accident': pd.Series(dtype='bool'),
        'Mass Shooting': pd.Series(dtype='bool')
    },
    index=['Incident ID']
)

In [132]:
# Concatenate the records from all the dataframes into a single final dataframe
for df in all_input_dfs:
    df.set_index('Incident ID', inplace=True, drop=False)
    all_victims_df.update(df, overwrite=False)
    all_victims_df = pd.concat([all_victims_df, df[~df.index.isin(all_victims_df.index)]])

In [133]:
# Remove duplicated header row
all_victims_df.dropna(inplace=True, how='all')

In [134]:
# Replace NAN values with False (the NAN values were intentional for making the
# update process easier)
all_victims_df.replace(np.nan, False, inplace=True)

In [135]:
# Make the Incident ID column an integer again
all_victims_df = all_victims_df.astype({'Incident ID': int})

In [136]:
# Sort the records by the Incident Date (ASC)
all_victims_df.sort_values(by='Incident Date', inplace=True)

In [137]:
# Break datetime column into multiple columns for the date dimension table
all_victims_df['Year'] = pd.DatetimeIndex(all_victims_df['Incident Date']).year
all_victims_df['Quarter'] = pd.DatetimeIndex(all_victims_df['Incident Date']).quarter
all_victims_df['Month'] = pd.DatetimeIndex(all_victims_df['Incident Date']).month
all_victims_df['Week'] = pd.DatetimeIndex(all_victims_df['Incident Date']).week
all_victims_df['Weekday'] = pd.DatetimeIndex(all_victims_df['Incident Date']).weekday

/var/folders/m_/nql1y1kj77z5ghk0gbbzt_180000gp/T/ipykernel_39337/2439861168.py:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  all_victims_df['Week'] = pd.DatetimeIndex(all_victims_df['Incident Date']).week


In [138]:
all_victims_df

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured,Children Involved,Teens Involved,Accident,Mass Shooting,Year,Quarter,Month,Week,Weekday
1967792,1967792,01-Apr-21,Louisiana,Baton Rouge,300 block of N Foster Dr,1.0,1.0,True,False,False,False,2021,2,4,13,3
1967420,1967420,01-Apr-21,West Virginia,Moorefield,506 Winchester Ave,3.0,0.0,True,False,False,False,2021,2,4,13,3
1966743,1966743,01-Apr-21,Ohio,Columbus,2500 block of Wallcrest Blvd,0.0,1.0,True,False,False,False,2021,2,4,13,3
1967044,1967044,01-Apr-21,Texas,Fort Worth,3200 block of Knox St,0.0,1.0,False,True,True,False,2021,2,4,13,3
1966396,1966396,01-Apr-21,Arizona,Phoenix,67th Ave and McDowell Rd,0.0,1.0,False,True,True,False,2021,2,4,13,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2156667,2156667,31-Oct-21,Arkansas,Little Rock,W 34th St,0.0,2.0,True,False,False,False,2021,4,10,43,6
2155897,2155897,31-Oct-21,Texas,Houston,7730 Moline St,1.0,1.0,False,True,False,False,2021,4,10,43,6
2155282,2155282,31-Oct-21,Texas,Houston,1909 Schilder Dr,1.0,3.0,False,True,False,True,2021,4,10,43,6
2155881,2155881,31-Oct-21,Missouri,Kansas City,E 54th St and Prospect Ave,1.0,0.0,False,True,False,False,2021,4,10,43,6


### Generate CSV from final dataframe

In [139]:
all_victims_df.to_csv('../output/all_victims_2021_to_date.csv')